In [3]:
from datetime import datetime, timedelta
from SQLiteFun import execution
from hapiclient import hapi
from time import sleep

# check and see if data can be retrieved from HAPI server
def DataChecker(server, dataset, start, stop, parameters, attempts):   
    # place in a try/except to wait x seconds and try again if this fails
    # if 3 fails in a row -> no data
    if attempts < 3:
        try:
            # Get data
            sleep(5.0)
            data, meta = hapi(server, dataset, parameters, start, stop)

        # if error arises
        except:
            print("Error raised, trying again")

        # if no error arises
        else:
            dataLen = len(data)
            print("The length of data is " + str(dataLen))

            # control loop
            dataFound = False
            # search for data
            while not dataFound:
                for entry in data:
                    print(type(data[0][1]))

                    # possible type mismatch when using boolean operator

                    # once data is found, end loop
                    if str(data[0][1]) != "":
                        dataFound = True
                        print("Data was successfully accessed")
                        print("Example data looks like " + str(data[i][1]))
                        break
        finally:
            attempts += 1
            return dataFound, attempts
    else:
        # insert code to input "HAPI info check passed after 1 attempt. HAPI data check failed after 3 attempts."
            #     into "Error" column in TestResults
        # code incorrect, needs to make use of INTERSECT to join the SELECT Spase_ids from MetadataEntries
            #     and Update commands since prodKey not a column in TestResults
        print("No data was found")
        HAPIErrorStmt = f""" UPDATE TestResults
                        SET Errors = 'HAPI info check passed after 1 attempt. HAPI data check failed after 3 attempts.'
                        WHERE prodKey = '{dataset}' """
        Record_id = execution(f""" SELECT rowNum FROM TestResults WHERE prodKey = '{dataset}';""")
        executionALL(authorStmt)
        print(f"Updated a TestResults entry with the row number {Record_id}")

# retrieve prodKeys associated with HAPI URLs
HapiStmt = """ SELECT prodKey FROM MetadataEntries WHERE url LIKE "%/hapi" LIMIT 10"""
prodKeys = execution(HapiStmt)
print("The prodKeys are " + str(prodKeys))


server = 'https://cdaweb.gsfc.nasa.gov/hapi'
#dataset    = 'JUNO_HELIO1DAY_POSITION'
#start      = '2011-08-06T00:00:00'
#stop       = '2011-08-06T00:30:00'
#parameters = 'SE_LAT'
#data, meta = hapi(server, dataset, parameters, start, stop)
#print(data)

# iterate thru prodKeys
for prodKey in prodKeys:
    # control loop
    dataFound = False
    # list that holds all parameters for a given server
    paramNames =  []

    dataset = str(prodKey)

    # retrieve all parameters and the start and stop date from the server
    meta = hapi(server,dataset)
    for k, v in meta.items():
        if k == "parameters":
            for params in v:
                for key, value in params.items():
                    if key == "name":
                        paramNames.append(value)
        elif k == "startDate":
            start = v
            date, sep, time = start.partition("T")
            time = time.replace("Z", "")
            dt_string = date + " " + time
            dt_obj = datetime.strptime(dt_string, "%Y-%m-%d %H:%M:%S")
            stop = dt_obj + timedelta(minutes=30)
            stop = str(stop)
            stop = stop.replace(" ", "T") + "Z"

    #print("Start time is " + start)
    #print("Stop time is " + stop)
    #print(paramNames)
    attempts = 0

    # iterate thru parameters in a server to see if any data can be pulled
    for parameters in paramNames[1:]:
        if not dataFound:
            print("Checking parameter " + str(parameters) + " in HAPI record with id " + dataset)
            dataFound, attempts = DataChecker(server, dataset, start, stop, parameters, attempts)


The prodKeys are ['JUNO_HELIO1DAY_POSITION', 'IA_K0_MFI', 'IA_K0_EPI', 'IA_K0_ENF', 'IA_OR_DEF', 'IA_K0_ICD', 'HELIOS1_COHO1HR_MERGED_MAG_PLASMA', 'HEL1_6SEC_NESSMAG', 'HELIOS1_HELIO1DAY_POSITION', 'HELIOS1_E6_1HOUR_PARTICLE_FLUX']
Checking parameter RAD_AU in HAPI record with id JUNO_HELIO1DAY_POSITION
The length of data is 1
<class 'numpy.float64'>
Data was successfully accessed
Example data looks like 1.0099999904632568
Checking parameter B_abs in HAPI record with id IA_K0_MFI
The length of data is 15
<class 'numpy.float64'>
Data was successfully accessed
Example data looks like 1608.0
Checking parameter Fe1 in HAPI record with id IA_K0_EPI
The length of data is 1
<class 'numpy.float64'>
Data was successfully accessed
Example data looks like -1e+31
Checking parameter E1 in HAPI record with id IA_K0_ENF
The length of data is 8
<class 'numpy.float64'>
Data was successfully accessed
Example data looks like 8.800000190734863
Checking parameter SC_pos_GSE in HAPI record with id IA_OR_DEF

NameError: name 'HAPIError' is not defined